In [2]:
library("dndscv")
library(dplyr)
library("biomaRt")
library(MASS) 
library(ggplot2)
library(lmtest)
library(stringr)
library(VennDiagram)
library(readr)
library(gridExtra)
library(RColorBrewer)
library(glue)
library(tidyr)
library(purrr)

In [4]:
patient_mut = 7000
gene_mut = 300

In [196]:
STAD_mutation = read_tsv("/Users/shuhaoxu/Desktop/BNFO285/Groupproject/Project2/TCGA.STAD.mutations.txt", quote="\"")
sample_mutation_counts <- table(STAD_mutation$patient_id)
gene_mutation_counts <- table(STAD_mutation$Hugo_Symbol)

# Filter hypermutated patients
print(nrow(STAD_mutation))
print(length(sample_mutation_counts))
hypermutators_patient <- names((sample_mutation_counts[(sample_mutation_counts > patient_mut) | (sample_mutation_counts <= 10)]))
print(length(hypermutators_patient))
STAD_mutation_filtered_patient <- STAD_mutation %>% filter(!patient_id %in% hypermutators_patient)
print(nrow(STAD_mutation_filtered_patient))

sample_mutation_counts_filtered_patient <- table(STAD_mutation_filtered_patient$patient_id)
gene_mutation_counts_filtered_patient <- table(STAD_mutation_filtered_patient$Hugo_Symbol)

# Filter out hyptermutated genes
gene_mutation_counts <- table(STAD_mutation$Hugo_Symbol)
print(length(gene_mutation_counts))
hypermutators_gene <- names((gene_mutation_counts[(gene_mutation_counts > gene_mut)]))
print(length(hypermutators_gene))
STAD_mutation_filtered_patient_gene <- STAD_mutation_filtered_patient %>% filter(!Hugo_Symbol %in% hypermutators_gene)
print(nrow(STAD_mutation_filtered_patient_gene))

STAD_mutation_filtered_pass <- STAD_mutation_filtered_patient_gene %>% filter(FILTER == "PASS")
nrow(STAD_mutation_filtered_pass)
STAD_mutation_filtered_pass <- STAD_mutation_filtered_pass[STAD_mutation_filtered_pass$VARIANT_CLASS == "SNV",]

Rows: 234941 Columns: 53
── Column specification ───────────────────────────────────────────────────────────────────
Delimiter: "\t"
chr (44): patient_id, Hugo_Symbol, Center, NCBI_Build, Chromosome, Strand, V...
dbl  (9): Entrez_Gene_Id, Start_Position, End_Position, t_depth, t_ref_count...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] 234941
[1] 415
[1] 9
[1] 203393
[1] 18933
[1] 2
[1] 202264


[1] 181529

In [7]:
ensembl <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")
transcript_ids <- unique(STAD_mutation_filtered_pass$Transcript_ID)

transcript_data <- getBM(attributes = c("ensembl_transcript_id", "hgnc_symbol", "transcript_length"),
                         filters = "ensembl_transcript_id",
                         values = transcript_ids,
                         mart = ensembl)

gene_lengths <- transcript_data %>%
  group_by(hgnc_symbol) %>%
  summarise(gene_length = max(transcript_length, na.rm = TRUE)) %>%
  ungroup()


In [197]:
mutation_summary <- STAD_mutation %>%
  group_by(Hugo_Symbol) %>%
  summarise(Samples_Mutated = n_distinct(patient_id)) %>%
  ungroup() %>%
  arrange(desc(Samples_Mutated)) %>%  
  mutate(Hugo_Symbol = factor(Hugo_Symbol, levels = Hugo_Symbol)) 

In [198]:
mutation_counts <- STAD_mutation_filtered_pass %>%
  group_by(Hugo_Symbol) %>%
  summarise(
    observed_mutations = n(),
    patients = list(unique(patient_id)) 
  ) %>%
  ungroup()

mutation_counts_with_GeneLength <- mutation_counts %>%
  left_join(gene_lengths, by = c("Hugo_Symbol" = "hgnc_symbol"))

mutation_counts_filtered <- mutation_counts_with_GeneLength %>%
  filter(!is.na(gene_length))

mutation_counts_filtered$log_gene_length <- log(mutation_counts_filtered$gene_length + 1e-6)

total_mutations <- sum(mutation_counts_filtered$observed_mutations)
total_length <- sum(mutation_counts_filtered$gene_length)
avg_mutation_rate <- total_mutations / total_length
cat("Average mutation rate (mutations per bp):", avg_mutation_rate, "\n")

mutation_counts_filtered <- mutation_counts_filtered %>%
  mutate(expected_mutations = gene_length * avg_mutation_rate)

STAD_mutation_filtered_pass <- STAD_mutation_filtered_pass %>%
mutate(
  SIFT_numeric = as.numeric(str_extract(SIFT, "(?<=\\()\\d*\\.?\\d*(?=\\))")),
  SIFT = ifelse(SIFT == ".", NA, SIFT)
)

impact_scores <- STAD_mutation_filtered_pass %>%
  group_by(Hugo_Symbol) %>%
  summarise(avg_sift = mean(SIFT_numeric, na.rm = TRUE)) %>%
  ungroup()

mutation_counts_filtered <- mutation_counts_filtered %>%
  left_join(impact_scores, by = "Hugo_Symbol")

mutation_counts_filtered$avg_sift <- replace(mutation_counts_filtered$avg_sift, is.na(mutation_counts_filtered$avg_sift), 0)

mutation_types <- STAD_mutation_filtered_pass %>%
  group_by(Hugo_Symbol) %>%
  summarise(
    dN = sum(Variant_Classification %in% c("Missense_Mutation", "Nonsense_Mutation", "Frame_Shift_Del", "Frame_Shift_Ins", "Splice_Site", "Translation_Start_Site")),
    dS = sum(Variant_Classification %in% c("Silent", "5'UTR", "Intron", "3'UTR", "3'Flank", "5'Flank"))
  ) %>%
  ungroup() %>%
  mutate(dN_dS = (dN / (dS + 1e-6)))

mutation_counts_filtered <- mutation_counts_filtered %>%
  left_join(mutation_types[, c("Hugo_Symbol", "dN_dS")], by = "Hugo_Symbol")

mutation_types <- STAD_mutation_filtered_pass %>%
  group_by(Hugo_Symbol) %>%
  summarise(
    dN = sum(Variant_Classification %in% c("Missense_Mutation", "Nonsense_Mutation", "Frame_Shift_Del", "Frame_Shift_Ins", "Splice_Site", "Translation_Start_Site")),
    dS = sum(Variant_Classification %in% c("Silent", "5'UTR", "Intron", "3'UTR", "3'Flank", "5'Flank"))
  ) %>%
  ungroup()

mutation_counts_filtered <- mutation_counts_filtered %>%
  left_join(mutation_types, by = "Hugo_Symbol")

t <- (sum(mutation_counts_filtered$dS) + sum(mutation_counts_filtered$dN)) / total_length

syn_by_substitution <- STAD_mutation_filtered_pass %>%
  filter(Variant_Classification %in% c("Silent", "5'UTR", "Intron", "3'UTR", "3'Flank", "5'Flank")) %>%
  mutate(substitution = paste(Reference_Allele, Tumor_Seq_Allele2, sep = ">")) %>%
  group_by(Hugo_Symbol, substitution) %>%
  summarise(count = n(), .groups = "drop") %>%
  pivot_wider(names_from = substitution, values_from = count, values_fill = 0, 
              names_prefix = "syn_")

dS_per_gene <- STAD_mutation_filtered_pass %>%
  filter(Variant_Classification %in% c("Silent", "5'UTR", "Intron", "3'UTR", "3'Flank", "5'Flank")) %>%
  group_by(Hugo_Symbol) %>%
  summarise(dS = n(), .groups = "drop")

mutation_counts_filtered <- mutation_counts_filtered %>%
  {if ("dS" %in% colnames(.)) dplyr::select(., -dS) else .} %>%
  left_join(dS_per_gene, by = "Hugo_Symbol") %>%
  left_join(syn_by_substitution, by = "Hugo_Symbol") %>%
  mutate(
    dS = coalesce(dS, 0),
    across(starts_with("syn_"), ~ coalesce(.x, 0))
  )
   
substitution_counts <- STAD_mutation_filtered_pass %>%
  mutate(substitution = paste(Reference_Allele, Tumor_Seq_Allele2, sep = ">")) %>%
  group_by(substitution) %>%
  summarise(count = n(), .groups = "drop")

total_counts <- sum(substitution_counts$count)

substitution_rates <- substitution_counts %>%
  mutate(rate = count / total_counts)
   
syn_long <- mutation_counts_filtered %>%
  dplyr::select(Hugo_Symbol, dplyr::starts_with("syn_")) %>%
  pivot_longer(cols = dplyr::starts_with("syn_"), names_to = "substitution", 
               values_to = "L_i_s", names_prefix = "syn_") %>%
  filter(L_i_s > 0)

syn_long_with_rates <- syn_long %>%
  left_join(substitution_rates %>% dplyr::select(substitution, r_i = rate), 
            by = "substitution")

syn_long_with_rates <- syn_long_with_rates %>%
  mutate(contribution = r_i * L_i_s)
   
expected_syn_per_gene <- syn_long_with_rates %>%
  group_by(Hugo_Symbol) %>%
  summarise(sum_contribution = sum(contribution), .groups = "drop") %>%
  mutate(expected_syn = t * sum_contribution)

mutation_counts_filtered <- mutation_counts_filtered %>%
  left_join(expected_syn_per_gene %>% dplyr::select(Hugo_Symbol, expected_syn), 
            by = "Hugo_Symbol")
   
mutation_counts_filtered <- mutation_counts_filtered %>%
  mutate(expected_syn = coalesce(expected_syn, 0))
   
impact_counts <- STAD_mutation_filtered_pass %>%
  group_by(Hugo_Symbol, IMPACT) %>%
  summarise(n = n(), .groups = 'drop') %>%
  pivot_wider(names_from = IMPACT, values_from = n, values_fill = 0) %>%
  rename(
    high_impact_count = HIGH,
    low_impact_count = LOW,
    moderate_impact_count = MODERATE,
    modifier_impact_count = MODIFIER
  )

mutation_counts_filtered <- mutation_counts_filtered %>%
  left_join(impact_counts, by = "Hugo_Symbol") %>%
  mutate(
    high_impact_count = replace_na(high_impact_count, 0),
    low_impact_count = replace_na(low_impact_count, 0),
    moderate_impact_count = replace_na(moderate_impact_count, 0),
    modifier_impact_count = replace_na(modifier_impact_count, 0)
  )

mutation_counts_filtered <- mutation_counts_filtered %>%
  mutate(offset2 = log(low_impact_count + moderate_impact_count + 1))

variant_counts <- STAD_mutation_filtered_pass %>%
  group_by(Hugo_Symbol, Variant_Classification) %>%
  summarise(n = n(), .groups = 'drop') %>%
  pivot_wider(names_from = Variant_Classification, values_from = n, values_fill = 0) %>%
  rename(
    count_3prime_flank = `3'Flank`,
    count_3prime_utr = `3'UTR`,
    count_5prime_flank = `5'Flank`,
    count_5prime_utr = `5'UTR`,
    count_intron = `Intron`,
    count_missense_mutation = `Missense_Mutation`,
    count_nonsense_mutation = `Nonsense_Mutation`,
    count_nonstop_mutation = `Nonstop_Mutation`,
    count_rna = `RNA`,
    count_silent = `Silent`,
    count_splice_site = `Splice_Site`,
    count_translation_start_site = `Translation_Start_Site`
  )

# Join the aggregated variant counts with mutation_counts_filtered
mutation_counts_filtered <- mutation_counts_filtered %>%
  left_join(variant_counts, by = "Hugo_Symbol") %>%
  # Replace NA with 0 for genes with no mutations of a particular variant type
  mutate(
    count_3prime_flank = replace_na(count_3prime_flank, 0),
    count_3prime_utr = replace_na(count_3prime_utr, 0),
    count_5prime_flank = replace_na(count_5prime_flank, 0),
    count_5prime_utr = replace_na(count_5prime_utr, 0),
    count_intron = replace_na(count_intron, 0),
    count_missense_mutation = replace_na(count_missense_mutation, 0),
    count_nonsense_mutation = replace_na(count_nonsense_mutation, 0),
    count_nonstop_mutation = replace_na(count_nonstop_mutation, 0),
    count_rna = replace_na(count_rna, 0),
    count_silent = replace_na(count_silent, 0),
    count_splice_site = replace_na(count_splice_site, 0),
    count_translation_start_site = replace_na(count_translation_start_site, 0)
  )


Average mutation rate (mutations per bp): 0.002399827 


In [22]:
cor.test(mutation_counts_filtered$observed_mutations, 
                              mutation_counts_filtered$modifier_impact_count, 
                              method = "pearson")


	Pearson's product-moment correlation

data:  mutation_counts_filtered$observed_mutations and mutation_counts_filtered$modifier_impact_count
t = 31.594, df = 15975, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.2278584 0.2570474
sample estimates:
      cor 
0.2425078 


In [21]:
cor.test(mutation_counts_filtered$observed_mutations, 
                              mutation_counts_filtered$moderate_impact_count, 
                              method = "pearson")


	Pearson's product-moment correlation

data:  mutation_counts_filtered$observed_mutations and mutation_counts_filtered$moderate_impact_count
t = 472.17, df = 15975, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.964938 0.967012
sample estimates:
      cor 
0.9659905 


In [19]:
cor.test(mutation_counts_filtered$observed_mutations, 
                              mutation_counts_filtered$high_impact_count, 
                              method = "pearson")


	Pearson's product-moment correlation

data:  mutation_counts_filtered$observed_mutations and mutation_counts_filtered$high_impact_count
t = 75.194, df = 15975, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.4997433 0.5226501
sample estimates:
      cor 
0.5112875 


In [20]:
cor.test(mutation_counts_filtered$observed_mutations, 
                              mutation_counts_filtered$low_impact_count, 
                              method = "pearson")


	Pearson's product-moment correlation

data:  mutation_counts_filtered$observed_mutations and mutation_counts_filtered$low_impact_count
t = 213.28, df = 15975, p-value < 2.2e-16
alternative hypothesis: true correlation is not equal to 0
95 percent confidence interval:
 0.8562014 0.8642632
sample estimates:
      cor 
0.8602861 


In [27]:
na_check <- sapply(mutation_counts_filtered[, c("observed_mutations", "log_gene_length", "avg_sift", "dN_dS", "modifier_impact_count", "low_impact_count", "moderate_impact_count")], function(x) any(is.na(x) | is.infinite(x)))
if (any(na_check)) {
  cat("Replacing NA/Inf with 0 or median where applicable...\n")
  mutation_counts_filtered <- mutation_counts_filtered %>%
    mutate(across(c(avg_sift, dN_dS, modifier_impact_count, low_impact_count, moderate_impact_count), 
                  ~ replace(., is.na(.) | is.infinite(.), 0))) %>%
    mutate(log_gene_length = replace(log_gene_length, is.na(log_gene_length) | is.infinite(log_gene_length), log(gene_length + 1e-6)))
}

In [32]:
# Null Model
null_model <- glm.nb(observed_mutations ~ 1 + offset(log_gene_length), 
                     data = mutation_counts_filtered)

# Alternative Models
combined_model <- glm.nb(observed_mutations ~ avg_sift + dN_dS + 
                         offset(log_gene_length) + 
                         offset(offset2), 
                         data = mutation_counts_filtered)

# Likelihood Ratio Tests
models <- list("Combined" = combined_model)

for (name in names(models)) {
  alt_model <- models[[name]]
  print(AIC(null_model, models[[name]]))
  lrt <- lrtest(null_model, alt_model)
  cat(sprintf("\nLikelihood Ratio Test: %s vs. Null\n", name))
  cat(sprintf("LR Statistic: %.10f, P-value: %.40f\n", lrt[2, "Chisq"], lrt[2, "Pr(>Chisq)"]))
}


dispersion_stat <- summary(combined_model)$deviance / summary(combined_model)$df.residual
cat("\nDispersion statistic for Combined Negative Model:", dispersion_stat, "\n")
if (dispersion_stat > 1.5) {
  cat("Overdispersion.\n")
}

               df      AIC
null_model      2 96159.84
models[[name]]  4 93536.84

Likelihood Ratio Test: Combined vs. Null
LR Statistic: 2627.0030950743, P-value: 0.0000000000000000000000000000000000000000

Dispersion statistic for Combined Negative Model: 1.181718 


# new feature separate each variant class / expression

In [38]:
variant_columns <- c(
  "count_3prime_flank",
  "count_3prime_utr",
  "count_5prime_flank",
  "count_5prime_utr",
  "count_intron",
  "count_missense_mutation",
  "count_nonsense_mutation",
  "count_nonstop_mutation",
  "count_rna",
  "count_silent",
  "count_splice_site",
  "count_translation_start_site"
)

# Initialize a data frame to store correlation results
correlation_results <- data.frame(
  Variant_Column = character(),
  Correlation_Coefficient = numeric(),
  P_Value = numeric(),
  stringsAsFactors = FALSE
)

# Loop through each variant column and compute Pearson correlation with observed_mutations
for (col in variant_columns) {
  # Compute Pearson correlation
  corr_test <- cor.test(mutation_counts_filtered$observed_mutations, 
                        mutation_counts_filtered[[col]], 
                        method = "pearson", 
                        use = "complete.obs")  # Handle NA values
  
  # Append results to the data frame
  correlation_results <- rbind(correlation_results, 
                              data.frame(
                                Variant_Column = col,
                                Correlation_Coefficient = corr_test$estimate,
                                P_Value = corr_test$p.value
                              ))
}

In [39]:
correlation_results

,Variant_Column,Correlation_Coefficient,P_Value
,<chr>,<dbl>,<dbl>
cor,count_3prime_flank,0.02165711,6.189503e-03
cor1,count_3prime_utr,0.12019980,1.713384e-52
cor2,count_5prime_flank,0.03772723,1.841771e-06
cor3,count_5prime_utr,0.11436858,1.165072e-47
cor4,count_intron,0.24906261,1.797413e-224
cor5,count_missense_mutation,0.96599054,0.000000e+00
cor6,count_nonsense_mutation,0.46613868,0.000000e+00
cor7,count_nonstop_mutation,0.01190767,1.323069e-01
cor8,count_rna,0.01345485,8.901086e-02


In [217]:
# Null Model
null_model <- glm.nb(dS ~ 1 + offset(log_gene_length) +
                         offset(offset2) + 
                         offset(log(count_translation_start_site + 1)) +
                         offset(log(count_rna + 1)) + 
                         offset(log(count_nonstop_mutation + 1)) +
                         offset(log(expected_syn + 1e-6)), 
                     data = mutation_counts_filtered)

# Alternative Models
combined_model <- glm.nb(dS ~ avg_sift + dN_dS + 
                         offset(offset2) + 
                         offset(log(count_translation_start_site + 1)) +
                         offset(log(count_rna + 1)) + 
                         offset(log(count_nonstop_mutation + 1)) +
                         offset(log(expected_syn + 1e-6)), 
                         data = mutation_counts_filtered)

# Likelihood Ratio Tests
models <- list("Combined" = combined_model)

for (name in names(models)) {
  alt_model <- models[[name]]
  print(AIC(null_model, models[[name]]))
  lrt <- lrtest(null_model, alt_model)
  cat(sprintf("\nLikelihood Ratio Test: %s vs. Null\n", name))
  cat(sprintf("LR Statistic: %.10f, P-value: %.40f\n", lrt[2, "Chisq"], lrt[2, "Pr(>Chisq)"]))
}


dispersion_stat <- summary(combined_model)$deviance / summary(combined_model)$df.residual
cat("\nDispersion statistic for Combined Negative Model:", dispersion_stat, "\n")
if (dispersion_stat > 1.5) {
  cat("Overdispersion.\n")
}

Warning message:
“glm.fit: algorithm did not converge”
Warning message in theta.ml(Y, mu, sum(w), w, limit = control$maxit, trace = control$trace > :
“iteration limit reached”
Warning message in sqrt(1/i):
“NaNs produced”
Warning message in glm.nb(dS ~ 1 + offset(log_gene_length) + offset(offset2) + offset(log(count_translation_start_site + :
“alternation limit reached”
Warning message:
“glm.fit: algorithm did not converge”
Warning message in glm.nb(dS ~ avg_sift + dN_dS + offset(offset2) + offset(log(count_translation_start_site + :
“alternation limit reached”


               df      AIC
null_model      2 74622.90
models[[name]]  4 62319.13

Likelihood Ratio Test: Combined vs. Null
LR Statistic: 12307.7688345593, P-value: 0.0000000000000000000000000000000000000000

Dispersion statistic for Combined Negative Model: 0.9206425 


In [111]:
library(tibble)

In [206]:
expression <- read_tsv("/Users/shuhaoxu/Desktop/BNFO285/Groupproject/Project2/TCGA.STAD.expression.txt", 
                       col_names = TRUE,
                       quote = "\"", 
                       show_col_types = FALSE)

colnames(expression) <- sapply(colnames(expression), function(x) strsplit(x, "\\|")[[1]][1])
expression <- expression[, !grepl("\\?", colnames(expression))]
names(expression) <- make.unique(names(expression))

In [215]:
compute_pca_scores <- function(gene, expr_data, mut_counts) {
  gene_patients <- unlist(mut_counts %>% filter(Hugo_Symbol == gene) %>% pull(patients))
  expression_subset <- expr_data %>%
    filter(patient_id %in% gene_patients) %>%
    dplyr::select(-patient_id)
  
  expression_t <- as.data.frame(expression_subset) %>%
    mutate_all(~replace(., is.na(.), 0))
  gene_vars <- apply(expression_t, 2, var, na.rm = TRUE)
  expression_t <- expression_t[, gene_vars > 0, drop = FALSE]
  
  if (nrow(expression_t) < 2) {
    return(matrix(NA, nrow = 1, ncol = 10)) 
  }

  pca_result <- prcomp(expression_t, scale. = TRUE, center = TRUE)
  pca_scores <- as.data.frame(pca_result$x[, 1:10])
  colnames(pca_scores) <- paste0("PC", 1:10)
  as.matrix(pca_scores)
}

In [213]:
mutation_counts_top10 <- mutation_counts %>% slice_head(n = 2)

In [216]:
cov_list <- map(
  mutation_counts_top10$Hugo_Symbol,
  ~ compute_pca_scores(.x, expression, mutation_counts_top10)
)

mutation_counts_top10 <- mutation_counts_top10 %>%
  mutate(expression_covariates = cov_list)

Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”


ERROR: [1m[33mError[39m in `map()`:[22m
[1m[22m[36mℹ[39m In index: 1.
[1mCaused by error in `[.data.frame`:[22m
[33m![39m undefined columns selected


In [168]:
mutation_counts_filtered <- mutation_counts_filtered %>%
  left_join(
    mutation_counts_top10 %>%
      dplyr::select(Hugo_Symbol, expression_covariates),
    by = "Hugo_Symbol"
  )

mutation_counts_test10 <- mutation_counts_filtered %>% slice_head(n = 2)

In [176]:
mutation_counts_test10$expression_mean <- sapply(mutation_counts_test10$expression_covariates, function(x) {
  if(is.list(x)) {
    mean(unlist(x), na.rm = TRUE)
  } else {
    mean(x, na.rm = TRUE)
  }
})

In [179]:
null_model <- glm.nb(observed_mutations ~ 1 + offset(log_gene_length) + offset(log(expected_syn + 1e-6)), 
                     data = mutation_counts_test10)

combined_model <- glm.nb(observed_mutations ~ expression_mean + offset(log_gene_length) + offset(log(expected_syn + 1e-6)), 
                         data = mutation_counts_test10)

# Likelihood Ratio Tests
models <- list("Combined" = combined_model)

for (name in names(models)) {
  alt_model <- models[[name]]
  print(AIC(null_model, models[[name]]))
  lrt <- lrtest(null_model, alt_model)
  cat(sprintf("\nLikelihood Ratio Test: %s vs. Null\n", name))
  cat(sprintf("LR Statistic: %.10f, P-value: %.40f\n", lrt[2, "Chisq"], lrt[2, "Pr(>Chisq)"]))
}


dispersion_stat <- summary(combined_model)$deviance / summary(combined_model)$df.residual
cat("\nDispersion statistic for Combined Negative Model:", dispersion_stat, "\n")
if (dispersion_stat > 1.5) {
  cat("Overdispersion.\n")
}

               df      AIC
null_model      2 18.35313
models[[name]]  3  6.00000

Likelihood Ratio Test: Combined vs. Null
LR Statistic: 14.3531341858, P-value: 0.0001515272412174538765351761160360410940

Dispersion statistic for Combined Negative Model: -Inf 
